## CREDIT SCORING PROJECT

**Цель исследования - проверить две гипотезы:**  
    1. Семейное положение клиента влияет на факт погашения кредита в срок  
    2. Количество детей клиента влияет на факт погашения кредита в срок

**Описание данных (из документации):**

children — количество детей в семье  
days_employed — общий трудовой стаж в днях   
dob_years — возраст клиента в годах   
education — уровень образования клиента   
education_id — идентификатор уровня образования   
family_status — семейное положение   
family_status_id — идентификатор семейного положения   
gender — пол клиента   
income_type — тип занятости   
debt — имел ли задолженность по возврату кредитов   
total_income — ежемесячный доход purpose — цель получения кредита


# План исследования: Анализ факторов, влияющих на возврат кредита в срок  
(Задача кредитного скоринга / классификации риска просрочки)

1. Загрузка и первичное изучение данных

- Открыть таблицу данных
- Изучить общую информацию о данных (размер, типы переменных, статистика describe(), info())
- Визуализировать базовые распределения (гистограммы, boxplot для числовых признаков)
- Определить целевую переменную (факт возврата кредита в срок / просрочки)  

2. Предобработка данных  

2.1 Удаление / обработка пропусков  

- Выявить долю и расположение пропусков (isnull().sum(), heatmap)
- Удаление строк/столбцов с критически большим количеством пропусков (если >70–80%)
- Обработка пропусков в количественных переменных (медиана / среднее / KNN-imputer)
- Обработка пропусков в категориальных переменных (мода / отдельная категория "unknown")

2.2 Обработка аномальных значений (выбросов)

- Выявление выбросов (boxplot, IQR-метод, z-score)
- Анализ природы аномалий (реальные экстремальные значения или ошибки ввода?)
- Принятие решения: удаление / замена на граничные значения / winsorization

2.3 Удаление / обработка дубликатов

- Поиск полных дубликатов строк (duplicated())
- Поиск дубликатов по ключевым полям (если есть ID клиента)
- Удаление / сохранение с пояснением

2.4 Изменение типов данных

- Приведение типов (int → category для небольших наборов значений, object → datetime)
- Корректное кодирование бинарных и порядковых признаков

2.5 Категоризация / биннинг признаков

- Преобразование непрерывных переменных в категории (возраст, доход, сумма кредита)
- Создание новых признаков (возрастные группы, категории дохода, долговая нагрузка и т.д.)

3. Исследовательский анализ данных (EDA) и ответы на ключевые вопросы

3.1 Исследование зависимостей от целевой переменной (возврат кредита в срок)

- 3.1.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?
- 3.1.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?
- 3.1.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- 3.1.4 Как разные цели кредита влияют на возврат кредита в срок?
- 3.1.5 Приведите возможные причины появления пропусков в исходных данных
- 3.1.6 Обоснуйте, почему заполнение пропусков медианным значением — лучшее решение для количественных переменных в данной задаче

(Для каждого пункта: группировки, pivot_table / crosstab, визуализации — barplot, countplot, boxplot по группам, статистические тесты при необходимости)

 4. Общий вывод по исследованию
  - Ключевые факторы, влияющие на возврат кредита
  - Сильные и слабые стороны данных
  

In [1]:
import pandas as pd

## 1. Загрузка и первичное изучение данных

- Открыть таблицу данных
- Изучить общую информацию о данных (размер, типы переменных, статистика describe(), info())
- Визуализировать базовые распределения (гистограммы, boxplot для числовых признаков)
- Определить целевую переменную (факт возврата кредита в срок / просрочки)

In [2]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
#выведем первые 20 строчек датафрейма data на экран
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
#выведем основную информацию о датафрейме с помощью метода info()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод:**

21 525 записей (RangeIndex: 0–21524)
12 столбцов
Типы данных:
int64: 5 столбцов (children, dob_years, education_id, family_status_id, debt)
float64: 2 столбца (days_employed, total_income)
object: 5 столбцов (education, family_status, gender, income_type, purpose)

Пропуски наблюдаются только в двух столбцах:
days_employed — 19351 non-null → ~2174 пропуска (~10.1%)
total_income — 19351 non-null → ~2174 пропуска (те же строки)

## 2. Предобработка данных

### 2.1 Удаление / обработка пропусков

- Выявить долю и расположение пропусков (isnull().sum(), heatmap)
- Удаление строк/столбцов с критически большим количеством пропусков (если >70–80%)
- Обработка пропусков в количественных переменных (медиана / среднее / KNN-imputer)
- Обработка пропусков в категориальных переменных (мода / отдельная категория "unknown")

In [5]:
#выведем количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — days_employed. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца income_type. Например, у человека с типом занятости сотрудник пропуск в столбце total_income должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [6]:
#пропуск в столбце total_income должен быть заполнен медианным доходом среди всех записей с тем же типом
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### 2.2 Обработка аномальных значений (выбросов)

- Выявление выбросов (boxplot, IQR-метод, z-score)
- Анализ природы аномалий (реальные экстремальные значения или ошибки ввода?)
- Принятие решения: удаление / замена на граничные значения / winsorization

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода abs().

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [8]:
#для каждого типа занятости выведите медианное значение трудового стажа days_employed в днях
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

In [9]:
#Выведем перечень уникальных значений столбца children
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

 В столбце children есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма data

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [11]:
#Выведем перечень уникальных значений столбца children, чтобы убедиться, что артефакты удалены
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### 2.3 Удаление / обработка дубликатов

- Поиск полных дубликатов строк (duplicated())
- Поиск дубликатов по ключевым полям (если есть ID клиента)
- Удаление / сохранение с пояснением


In [12]:
#Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
#Убедитесь, что все пропуски заполнены. 
#ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### 2.4 Изменение типов данных

- Приведение типов (int → category для небольших наборов значений, object → datetime)
- Корректное кодирование бинарных и порядковых признаков

In [14]:
# Заменим вещественный тип данных в столбце total_income на целочисленный с помощью метода astype()
data['total_income'] = data['total_income'].astype(int)

### 2.5 Категоризация / биннинг признаков

- Преобразование непрерывных переменных в категории (возраст, доход, сумма кредита)
- Создание новых признаков (возрастные группы, категории дохода, долговая нагрузка и т.д.)

Обработаем неявные дубликаты в столбце education. 
В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. 
Приведем их к нижнему регистру. Проверем остальные столбцы

In [15]:
data['education'] = data['education'].str.lower()

In [16]:
#Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

#### 2.5.1 Категоризация данных
На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:  

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`.   
Используем собственную функцию с именем `categorize_income()` и метод `apply()`

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [20]:
#Выведем на экран перечень уникальных целей взятия кредита из столбца purpose
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, 
в который войдут следующие категории:  
'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'.  
Например, если в столбце purpose находится подстрока 'на покупку автомобиля', то в столбце purpose_category должна появиться строка 'операции с автомобилем'.

Используем собственную функцию с именем categorize_purpose() и метод apply(). Изучим данные в столбце purpose и определим, какие подстроки помогут правильно определить категорию.

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [23]:
# Сколько кредитов в каждой категории целей
print("Распределение целей кредита:")
display(data['purpose_category'].value_counts())

# В процентах, % (удобно для выводов)
print("\nВ процентах, %:")
display(round(data['purpose_category'].value_counts(normalize=True) * 100, 2))

Распределение целей кредита:


purpose_category
операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: count, dtype: int64


В процентах, %:


purpose_category
операции с недвижимостью    50.40
операции с автомобилем      20.06
получение образования       18.70
проведение свадьбы          10.84
Name: proportion, dtype: float64

## 3. Исследовательский анализ данных (EDA) и ответы на ключевые вопросы

### 3.1 Исследование зависимостей от целевой переменной (возврат кредита в срок)

- 3.1.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?
- 3.1.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?
- 3.1.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- 3.1.4 Как разные цели кредита влияют на возврат кредита в срок?
- 3.1.5 Приведите возможные причины появления пропусков в исходных данных
- 3.1.6 Обоснуйте, почему заполнение пропусков медианным значением — лучшее решение для количественных переменных в данной задаче

(Для каждого пункта: группировки, pivot_table / crosstab, визуализации — barplot, countplot, boxplot по группам, статистические тесты при необходимости)


#### 3.1.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [24]:
# Категоризируем заемщиков по числку детей
def categorize_children(children):
    if children == 0:
        return 'без детей'
    elif children == 1:
        return '1 ребенок'
    elif children == 2:
        return '2 детей'
    else:
        return 'многодетные'

In [25]:
data['children_category'] = data['children'].apply(categorize_children)

In [26]:
#Рассчитаем процент возврата кредита в срок для каждой категории детей
grouped_children = data.groupby('children_category')['debt'].mean() * 100
grouped_children

children_category
1 ребенок      9.234609
2 детей        9.454191
без детей      7.543822
многодетные    8.157895
Name: debt, dtype: float64

In [27]:
# Создаем сводную таблицу
pivot_table_children = data.pivot_table(index='children_category', values='debt', aggfunc=['count', 'sum'])

# Переименуем столбцы 
pivot_table_children.columns = ['total_clients', 'debtors']

# Рассчитываем долю просрочки
pivot_table_children['debt_percentage'] = (pivot_table_children['debtors'] / pivot_table_children['total_clients']) * 100
pivot_table_children

,total_clients,debtors,debt_percentage
children_category,,,
1 ребенок,4808,444,9.234609
2 детей,2052,194,9.454191
без детей,14091,1063,7.543822
многодетные,380,31,8.157895


**Вывод:** Процент возврата кредита у категории заемщиков 'без бетей' ниже, чем у заемщиков с одним или двумя детьми. 
Стоит отметить - многодетные заемщики возвращают кредит в срок чаще чем заемщики без детей. В целом процентное соотношение отличается несущественно, выделить сильно выраженную зависимость возврата кредита от количества детей нельзя.
  
Анализируя данные, мы обратили внимание на небольшое различие в доле возврата кредита в срок в зависимости от наличия детей у заемщиков. После вывода сводной таблицы можно отметить, что выборки не сбалансированы: количество заемщиков с детьми значительно меньше, чем бездетных заемщиков.

С учетом этого неравенства в выборках, стоит аккуратно интерпретировать результаты. Несмотря на небольшое различие в процентах, оно может иметь статистическое значение из-за большой общей численности бездетных заемщиков.

Таким образом, можно предварительно сказать, что наличие детей влияет на возврат кредита в срок, но для более точных и надежных выводов необходимо провести более глубокий статистический анализ, возможно, с учетом других переменных, чтобы подтвердить или опровергнуть данное предположение.

#### 3.1.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [28]:
# Рассчитаем процент возврата кредита в срок для каждой категории семейного положения
grouped_family_status = data.groupby('family_status')['debt'].mean() * 100
grouped_family_status 

family_status
Не женат / не замужем    9.763948
в разводе                7.064760
вдовец / вдова           6.624606
гражданский брак         9.313014
женат / замужем          7.560558
Name: debt, dtype: float64

In [29]:
# Создаем сводную таблицу
pivot_table_family_status = data.pivot_table(index='family_status', values='debt', aggfunc=['count', 'sum'])

# Переименуем столбцы 
pivot_table_family_status.columns = ['total_clients', 'debtors']

# Рассчитываем долю просрочки
pivot_table_family_status['debt_percentage'] = (pivot_table_family_status['debtors'] / pivot_table_family_status['total_clients']) * 100
pivot_table_family_status

,total_clients,debtors,debt_percentage
family_status,,,
Не женат / не замужем,2796,273,9.763948
в разводе,1189,84,7.064760
вдовец / вдова,951,63,6.624606
гражданский брак,4134,385,9.313014
женат / замужем,12261,927,7.560558


**Вывод:**  Результат показывает процент возврата кредита в срок для каждой категории семейного положения. Если процент возврата кредита в срок выше или ниже среднего значения в наборе данных, это может указывать на наличие зависимости между семейным положением и возвратом кредита в срок. Уатегории заемщиков 'вдовец/вдова' процент возврата кредита вовремя самый низкий и почти в 1,5 раза отличается от категорий 'не женат/не замужем' или 'гражданский брак', в которых процент возврата кредита самый высокий.  Можно сделать вывод, что существует некоторая зависимость между уровнем дохода и возвратом кредита в срок.  

После вывода сводной таблицы можно отметить, что выборки не сбалансированы: суммарное количество заемщиков не состоящих в официальном браке, овдовевших и разведенных меньше, чем заемщиков в категории 'женат/замужем'.

С учетом этого неравенства в выборках, стоит аккуратно интерпретировать результаты. Несмотря на небольшое различие в процентах, оно может иметь статистическое значение из-за большой общей численности заемщиков состоящих в официальном браке.

Таким образом, можно предварительно сказать, что что заемщики состоящие или состоявшие в браке выглядят более надежными

#### 3.1.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [30]:
# Уровни дохода уже категоризированы в пункте 2.5
# Рассчитаем процент возврата кредита внутри каждой категории
grouped_income = data.groupby('total_income_category')['debt'].mean() * 100
grouped_income

total_income_category
A    8.000000
B    7.060231
C    8.498210
D    6.017192
E    9.090909
Name: debt, dtype: float64

In [31]:
# Создаем сводную таблицу
pivot_table_income = data.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'sum'])
# Переименовываем столбцы
pivot_table_income.columns = ['total_clients', 'debtors']
# Добавляем столбец с долей просрочки
pivot_table_income['debt_percentage'] = (pivot_table_income['debtors'] / pivot_table_income['total_clients']) * 100
# Сортируем таблицу по доле просрочки по убыванию
pivot_table_income = pivot_table_income.sort_values(by='debt_percentage', ascending=False)
pivot_table_income

,total_clients,debtors,debt_percentage
total_income_category,,,
E,22,2,9.090909
C,15921,1353,8.498210
A,25,2,8.000000
B,5014,354,7.060231
D,349,21,6.017192


**Вывод:** 
На основании анализа данных о возврате кредитов в срок в зависимости от уровня дохода, можно сделать следующие выводы:

Категория 'A' (доходы 1,000,001 и выше): Процент возврата кредита составляет 8.00%. Это говорит о том, что люди с очень высоким уровнем дохода имеют относительно низкии процент долгов.

Категория 'B' (доходы от 200,001 до 1,000,000): Процент возврата в этой категории составляет 7.06%. Это также низкий процент, что может свидетельствовать о том, что люди с высокими доходами в целом довольно ответственны в вопросах возврата кредитов.

Категория 'C' (доходы от 50,001 до 200,000): Процент возврата кредита в этой категории составляет 8.50%. Это средний уровень, который может говорить о том, что люди с умеренным доходом возвращают кредиты в срок чуть лучше, чем средний показатель.

Категория 'D' (доходы от 30,001 до 50,000): Процент возврата в этой категории составляет 6.02%. Это самый низкий процент среди всех категорий, что может свидетельсивовать о том, что люди с низкими средними доходами имеют трудности с возвратом кредитов.

Категория 'E' (доходы до 30,000): Процент возврата в этой категории составляет 9.09%. Это самый высокий процент возврата, что может говорить о том, что люди с самыми низкими доходами возвращают кредиты в срок чаще других категорий.

Таким образом, можно сделать вывод, что существует некоторая зависимость между уровнем дохода и возвратом кредита в срок. Люди с более высокими доходами имеют тенденцию чаще возвращать кредиты вовремя, в то время как уровень надежности снижается с уменьшением доходов.

#### 3.1.4 Как разные цели кредита влияют на возврат кредита в срок?

In [32]:
# Рассчитаем процент возврата для каждой категории цели кредита
purpose_debt_percentage = data.groupby('purpose_category')['debt'].mean() * 100
purpose_debt_percentage

purpose_category
операции с автомобилем      9.347978
операции с недвижимостью    7.255139
получение образования       9.252758
проведение свадьбы          7.911803
Name: debt, dtype: float64

In [33]:
# Создаем сводную таблицу
pivot_table_purpose_category = data.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum'])

# Переименуем столбцы 
pivot_table_purpose_category.columns = ['total_clients', 'debtors']

# Рассчитываем долю просрочки
pivot_table_purpose_category['debt_percentage'] = (pivot_table_purpose_category['debtors'] / pivot_table_purpose_category['total_clients']) * 100
pivot_table_purpose_category

,total_clients,debtors,debt_percentage
purpose_category,,,
операции с автомобилем,4279,400,9.347978
операции с недвижимостью,10751,780,7.255139
получение образования,3988,369,9.252758
проведение свадьбы,2313,183,7.911803


**Вывод:** 

Операции с автомобилем: процент возврата кредита для клиентов, берущих кредит на операции с автомобилем, составляет 9.35%. Это означает, что почти 91.65% клиентов успешно возвращают кредит в срок, что делает эту категорию наиболее надежной в аспекте возврата.

Операции с недвижимостью: клиенты, берущие кредит на операции с недвижимостью, имеют процент возврата кредита в размере 7.26%. Это говорит о том, что 92.74% клиентов возвращают кредит в срок, что является довольно высоким показателем.

Получение образования: процент возврата кредита для клиентов, берущих кредит на получение образования, составляет 9.25%. Большинство клиентов (90.75%) успешно возвращают кредиты в данной категории.

Проведение свадьбы: клиенты, берущие кредит на проведение свадьбы, имеют процент возврата кредита в размере 7.91%. Это означает, что 92.09% клиентов возвращают кредит в срок.

Исходя из этих данных, можно сказать, что клиенты, берущие кредит на операции с недвижимостью и проведение свадьбы, обычно более надежные заемщики, чем те, кто берет кредит на образование или автомобиль. Однако разница в процентах возврата не слишком велика между различными категориями целей кредита.

#### 3.1.5 Возможные причины появления пропусков в исходных данных

Причины появления пропуска в данных:

1.Человеческий фактор: ошибка ввода данных при внесении информации в систему.

2.Отсутствие информации: некоторые анкетные данные могут быть пропущены из-за отсутствия ответа от клиента или необходимости предоставить определенные сведения (например, клиент может не хотеть указывать свой доход или семейное положение по личным убеждениям или из соображений конфиденциальности).

3.Технические проблемы: системные сбои или ошибки при выгрузке данных из базы данных 

4.Ошибка при передаче данных: При передаче данных между системами или при внесении данных вручную могут возникнуть ошибки, приводящие к пропускам.

#### 3.1.6 Почему заполнение пропусков медианным значением — лучшее решение для количественных переменных в данной задаче

Заполнение пропусков медианным значением является одним из распространенных методов обработки отсутствующих данных в количественных переменных. Это решение часто используется по нескольким причинам:

1.Сохранение центральной тенденции: медиана является мерой центральной тенденции, которая не чувствительна к выбросам. Если в данных есть аномальные значения (выбросы), среднее значение может исказиться, в то время как медиана будет более устойчива к ним.

2.Подходит для скошенных распределений: если распределение данных смещено, медиана может быть более репрезентативной мерой центральной тенденции, чем среднее значение.

3.Простота и эффективность: замена отсутствующих значений медианой легко реализуется и не требует сложных вычислений. Кроме того, это эффективный способ обработки больших наборов данных.


## 4. Общий вывод 

О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 
Изучение общей информации о данных таблицы, предоставленной банком, показало  что количество значений в столбцах различается, есть пропущенные данные в столбцах total_income и days_employed. Тк это количественные данные, заполнили пропущенные значения медианным значением. Это решение позволяет сохранить все строки и 
Были обработаны аномальные значения которые могут повлиять на анлиз данных: отрицательное количество детей и отрицательный трудовой стаж. Обработаны неявные дубликаты, и категоризирован перечень уникальных целей.
Эта предварительная подготовка данных позволила получить более чистый и структурированный набор данных для анализа. 
Имея подготовленные данные, можно провести более детальный анализ и проверить различные гипотезы для выявления важных зависимостей и паттернов в данных.

В результате анализа предоставленных данных о платежеспособности клиентов можно сделать следующие выводы:

- Дети и возврат кредита: 
Заемщики с детьми имеют немного больший процент невозврата кредита. Однако, эта разница невелика и не позволяет делать однозначные выводы о зависимости возврата кредита от наличия детей.

- Семейное положение и возврат кредита:
Люди в браке или находящиеся в гражданском браке возвращают кредиты лучше, чем неженатые, разведенные или овдовевшие заемщики.

- Уровень дохода и возврат кредита: 
Заемщики с более высоким уровнем дохода возвращают кредиты надежнее. Однако, интересно, что категория заемщиков с уровнем дохода от 30,000 до 50,000 также показывает надежность в возврате кредитов, возможно, из-за их большего количества.

- Цель кредита и возврат кредита: 
Люди, берущие кредит на образование или автомобиль, возвращают его чаще, чем те, кто берет кредит на свадьбу или недвижимость.

Исходя из этих выводов, банк может скорректировать свои стратегии выдачи кредитов. Например, уделить больше внимания клиентам с детьми, предоставлять более гибкие условия для заемщиков в определенным уровнем дохода, учесть цель кредита при определении процентной ставки и сроков возврата. Также важно продолжить мониторить и анализировать данные для выявления новых закономерностей и трендов в поведении заемщиков и обратить внимание сотрудников банка на наличие пропусков. Пропуски данных могут вносить искажения в анализ. Для более точного анализа данных при возможности, заполнять их наиболее точными данными.
